In [35]:
%load_ext autoreload
%autoreload 2


#@markdown ### Librerías y funciones auxiliares
import sys
import os

import numpy as np
from dotenv import load_dotenv
from google.cloud import storage
from tqdm import trange

import pandas as pd
from sklearn.cluster import DBSCAN, HDBSCAN
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.ensemble import IsolationForest
from sklearn.neighbors import LocalOutlierFactor

    
# Añadir el directorio raíz del proyecto al sys.path
project_path = os.path.abspath(os.path.join(os.getcwd(), "../../src"))  # Subir un nivel
if project_path not in sys.path:
    sys.path.append(project_path)
            
from hyper_velocity_stars_detection.jobs.utils import read_clusters_harris_catalog
from hyper_velocity_stars_detection.jobs.google_jobs.utils import load_globular_cluster, ProjectDontExist
from hyper_velocity_stars_detection.cluster_detection.clustering_methods import GaussianMixtureClustering

load_dotenv("../../data/.env")
os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = "../../data/hvs-storage.json"

PATH = "../../data/report_notebook"

RADIUS_SCALE = 1
CATALOG = "gaiadr3"
FILTERS = {"ast_params_solved": 3, "ruwe": 1.4, "v_periods_used": 10, "min_parallax": 0}
PROJECT = os.environ["PROJECT_ID"]
BUCKET = os.environ["BUCKET"]
BUCKET_PATH = "report/gc_clusters/"

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [36]:
clusters_specials = ["ngc 104", "ngc 5139", "ngc 5286", "ngc 6266", "ngc 6388"]
clusters = read_clusters_harris_catalog()
gc_objects = {}
for pos in trange(len(clusters)):
    cluster = clusters[pos].name
    try:
        gc_objects[cluster] = load_globular_cluster(cluster, PROJECT, BUCKET, BUCKET_PATH)
    except ProjectDontExist:
        continue


100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 147/147 [03:06<00:00,  1.27s/it]


In [27]:
def get_algorithm_type(gc_object: GaussianMixtureClustering) -> str:
    model = gc.clustering_results.clustering.model
    if isinstance(model, HDBSCAN):
        return "HDBSCAN"
    if isinstance(model, DBSCAN):
        return "DBSCAN"
    if isinstance(model, GaussianMixtureClustering):
        return "GaussianMixtureModel"
    return None

def get_noise_method(gc_object: GaussianMixtureClustering) -> str:
    noise_method = gc.clustering_results.clustering.noise_method
    if noise_method is not None:
        model = noise_method.model
        if isinstance(model, LocalOutlierFactor):
            return "LocalOutlierFactor"
        if isinstance(model, IsolationForest):
            return "IsolationForest"
    return None


def get_scaler(gc_object: GaussianMixtureClustering) -> str:
    scaler = gc.clustering_results.clustering.scaler
    if scaler is not None:
        if isinstance(scaler, StandardScaler):
            return "StandardScaler"
        if isinstance(scaler, MinMaxScaler):
            return "MinMaxScaler"
    return None

In [40]:
results = pd.DataFrame(columns=["Name", "Cluster Algorithm", "Noise Method", "Scaler"])

for pos, cluster in enumerate(gc_objects.keys()):
    gc = gc_objects[cluster]
    algorithm, noise, scaler = get_algorithm_type(gc), get_noise_method(gc), get_scaler(gc)
    results.loc[pos] = (gc.name,algorithm, noise, scaler)
results

,Name,Cluster Algorithm,Noise Method,Scaler
0,NGC_104,HDBSCAN,None,StandardScaler
1,NGC_288,GaussianMixtureModel,IsolationForest,StandardScaler
2,NGC_362,HDBSCAN,None,StandardScaler
3,NGC_1261,HDBSCAN,None,StandardScaler
4,NAME_E_1,DBSCAN,None,StandardScaler
...,...,...,...,...
115,M_15,HDBSCAN,None,StandardScaler
116,M_2,DBSCAN,None,StandardScaler
117,M_30,DBSCAN,None,StandardScaler
118,Cl_Pal_12,GaussianMixtureModel,IsolationForest,MinMaxScaler


In [45]:
results.groupby(["Cluster Algorithm", "Noise Method"]).count()

Name  Scaler
Cluster Algorithm    Noise Method                    
GaussianMixtureModel IsolationForest       12      12
                     LocalOutlierFactor     4       3

In [42]:
results["Noise Method"].value_counts()


Noise Method
IsolationForest       12
LocalOutlierFactor     4
Name: count, dtype: int64

In [43]:
results["Scaler"].value_counts()

Scaler
StandardScaler    112
MinMaxScaler        7
Name: count, dtype: int64

In [46]:
results_special = pd.DataFrame(columns=["Name", "Cluster Algorithm", "Noise Method", "Scaler"])

for pos, cluster in enumerate(clusters_specials):
    gc = gc_objects[cluster]
    algorithm, noise, scaler = get_algorithm_type(gc), get_noise_method(gc), get_scaler(gc)
    results_special.loc[pos] = (gc.name,algorithm, noise, scaler)
results_special

,Name,Cluster Algorithm,Noise Method,Scaler
0,NGC_104,HDBSCAN,None,StandardScaler
1,NGC_5139,HDBSCAN,None,StandardScaler
2,NGC_5286,HDBSCAN,None,StandardScaler
3,M_62,HDBSCAN,None,StandardScaler
4,NGC_6388,HDBSCAN,None,StandardScaler


In [51]:
gc = gc_objects["ngc 104"]
df_stars = gc.clustering_results.remove_outliers_gc()
df_stars

,solution_id,DESIGNATION,SOURCE_ID,random_index,ref_epoch,ra,ra_error,dec,dec_error,parallax,...,ebpminrp_gspphot,ebpminrp_gspphot_lower,ebpminrp_gspphot_upper,libname_gspphot,parallax_orig,pmra_kms,pmdec_kms,pm_kms,pm_l,pm_b
6,1636148068921376768,Gaia DR3 4689578583242455936,4689578583242455936,1130474818,2016.0,7.338457,0.009057,-72.185311,0.008556,0.220200,...,NaN,NaN,NaN,NaN,0.199180,109.562839,-45.381889,118.589762,-5.302953,1.477005
14,1636148068921376768,Gaia DR3 4689877654695832704,4689877654695832704,22873249,2016.0,5.304468,0.009274,-71.640867,0.008672,0.224504,...,NaN,NaN,NaN,NaN,0.220162,110.725610,-56.294604,124.214504,-5.610366,1.755534
22,1636148068921376768,Gaia DR3 4689631497239536896,4689631497239536896,841749339,2016.0,6.904847,0.009456,-72.061879,0.008914,0.241363,...,0.1946,0.1923,0.1964,MARCS,0.215629,98.627490,-48.176705,109.765098,-5.295063,1.775669
23,1636148068921376768,Gaia DR3 4689618891515871744,4689618891515871744,159225896,2016.0,5.360483,0.009087,-72.091151,0.008593,0.245425,...,0.2025,0.2010,0.2040,MARCS,0.212621,98.140186,-54.181412,112.103173,-5.468050,1.933455
28,1636148068921376768,Gaia DR3 4689638476571141248,4689638476571141248,362061088,2016.0,5.623070,0.008864,-72.099117,0.008908,0.227974,...,0.0710,0.0333,0.0777,PHOENIX,0.195417,109.810103,-60.857299,125.546284,-5.673323,2.053345
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4991,1636148068921376768,Gaia DR3 4689832196762182016,4689832196762182016,563905293,2016.0,5.794131,0.039524,-71.905779,0.042189,0.214442,...,0.0700,0.0557,0.0899,MARCS,0.184866,99.600731,-59.122716,115.826599,-4.862224,1.942695
4995,1636148068921376768,Gaia DR3 4689647543252194816,4689647543252194816,861269635,2016.0,6.693628,0.041879,-71.886442,0.041130,0.274218,...,0.0224,0.0142,0.0311,MARCS,0.244763,86.317636,-42.417342,96.176739,-5.275830,1.753533
4996,1636148068921376768,Gaia DR3 4689637956871213568,4689637956871213568,946113673,2016.0,5.905170,0.038069,-72.104726,0.039758,0.224046,...,NaN,NaN,NaN,NaN,0.187959,126.370689,-60.375924,140.052859,-6.332570,1.911048
4997,1636148068921376768,Gaia DR3 4689620192879865728,4689620192879865728,581982837,2016.0,6.051463,0.042554,-72.281971,0.039577,0.271411,...,0.0018,0.0004,0.0053,MARCS,0.241778,95.891260,-40.356510,104.037405,-5.768287,1.469195


In [88]:
pm_kms = np.sqrt((df_stars.pmra_kms - df_stars.pmra_kms.mean())**2 + (df_stars.pmdec_kms - df_stars.pmdec_kms.mean())**2)
v_kms =  np.sqrt(pm_kms**2 + (df_stars.radial_velocity - df_stars.radial_velocity.mean())**2)
v_kms.std()

np.float64(8.315807575199118)

In [110]:
sigma_47tuc = v_kms.std()
2 * ((sigma_47tuc / 70)**4 )*1e6

np.float64(398.3415060698599)

In [111]:
sigma_47tuc = v_kms.std()
sigma_200 = sigma_47tuc / 200
mean = 7.41 + 3.28 * np.log10(sigma_200)
lower = 8.32-0.04 + (5.35 + 0.23) * np.log10(sigma_200)
upper = 8.32+0.04 + (5.35 - 0.23) * np.log10(sigma_200)

In [119]:
df_mac[df_mac.Name == "NGC 288"]

,recno,Cluster,RAJ2000,DEJ2000,SB,M,D,SName,logAge,__Fe_H_,...,Ltot,e_Ltot,E_Ltot,Mtot,e_Mtot,E_Mtot,M_L,e_M_L,E_M_L,Name


In [62]:
# Create a Python script that estimates IMBH mass using two scaling relations
# (M-σ and M_cluster–M_BH from Lützgendorf+2013) and then computes the expected
# HVS ejection speed distribution using the Hills-type scaling (Fragione & Gualandris style).
#
# We'll also generate a small demo table (including a 47 Tuc-like example) and save it.

import numpy as np
import pandas as pd
from dataclasses import dataclass
from typing import Tuple, Dict, Sequence, Optional

# -----------------------------
# Scaling relations (literature)
# -----------------------------
# All logarithms are base-10. Scatter terms are intrinsic scatter in dex.
#
# 1) Galaxy M-σ (McConnell & Ma 2013):
#    log10(M_BH/Msun) = alpha + beta * log10(sigma / 200 km/s)
MCMA_ALPHA = 8.32
MCMA_BETA = 5.64
MCMA_SIGMA_DEX = 0.38  # typical intrinsic scatter ~0.38 dex (literature range 0.30–0.45)

# 2) IMBH in GCs (Lützgendorf+2013; EM algorithm; Table 3):
#    log10(M_BH/Msun) = alpha + beta * log10(sigma / 200 km/s)
L13_SIGMA_ALPHA = 7.41
L13_SIGMA_BETA = 3.28
L13_SIGMA_SCATTER_DEX = 0.36

#    log10(M_BH/Msun) = alpha + beta * log10(M_tot / 1e11 Msun)
L13_MASS_ALPHA = 8.63
L13_MASS_BETA = 1.01
L13_MASS_SCATTER_DEX = 0.49

# -----------------------------
# Ejection speed scaling (Hills-like)
# v_ej ≈ 460 km/s * (a/0.1 AU)^(-1/2) * (m_tot/2 Msun)^(1/3) * (M_IMBH/1e3 Msun)^(1/6)
# We'll model extra fractional scatter on v_ej of ~30%.
# -----------------------------

def v_ej_median(a_AU: float, m_tot_Msun: float, M_IMBH_Msun: float) -> float:
    return 460.0 * (a_AU / 0.1) ** (-0.5) * (m_tot_Msun / 2.0) ** (1.0/3.0) * (M_IMBH_Msun / 1.0e3) ** (1.0/6.0)

# Utility to convert dex scatter to natural-log scatter
def dex_to_ln(s_dex: float) -> float:
    return s_dex * np.log(10.0)

@dataclass
class MBHEstimate:
    log10_MBH: float
    scatter_dex: float
    source: str

def estimate_mbh_from_sigma(sigma_kms: float, method: str = "L13") -> MBHEstimate:
    if sigma_kms <= 0:
        raise ValueError("sigma must be positive.")
    if method.upper() == "MCMA13":
        alpha, beta, s = MCMA_ALPHA, MCMA_BETA, MCMA_SIGMA_DEX
        src = "McConnell & Ma 2013 (galaxies)"
    else:
        alpha, beta, s = L13_SIGMA_ALPHA, L13_SIGMA_BETA, L13_SIGMA_SCATTER_DEX
        src = "Lützgendorf+2013 EM (GC IMBH)"
    x = np.log10(sigma_kms / 200.0)
    log10_mbh = alpha + beta * x
    return MBHEstimate(log10_mbh, s, src)

def estimate_mbh_from_mcluster(M_cluster_Msun: float, method: str = "L13") -> MBHEstimate:
    if M_cluster_Msun <= 0:
        raise ValueError("Cluster mass must be positive.")
    # Only Lützgendorf+2013 provides M_BH–M_cluster in GC context
    alpha, beta, s = L13_MASS_ALPHA, L13_MASS_BETA, L13_MASS_SCATTER_DEX
    src = "Lützgendorf+2013 EM (GC IMBH)"
    x = np.log10(M_cluster_Msun / 1.0e11)
    log10_mbh = alpha + beta * x
    return MBHEstimate(log10_mbh, s, src)

def sample_mbh(m_est: MBHEstimate, nsamp: int = 20000, seed: Optional[int] = 123) -> np.ndarray:
    rng = np.random.default_rng(seed)
    # Lognormal in base-10: sample in natural log-space
    mu_ln = m_est.log10_MBH * np.log(10.0)
    sigma_ln = dex_to_ln(m_est.scatter_dex)
    ln_samples = rng.normal(loc=mu_ln, scale=sigma_ln, size=nsamp)
    return np.exp(ln_samples)  # Msun

def summarize(arr: np.ndarray, qs: Sequence[float] = (0.16, 0.50, 0.84)) -> Dict[str, float]:
    qv = np.quantile(arr, qs)
    return {f"q{int(q*100):02d}": float(v) for q, v in zip(qs, qv)}

def combine_to_vej(sigma_kms: Optional[float] = None,
                   M_cluster_Msun: Optional[float] = None,
                   method_sigma: str = "L13",
                   a_list_AU: Sequence[float] = (0.05, 0.10, 0.20, 0.50),
                   m1: float = 0.8,
                   m2: float = 0.8,
                   v_scatter_frac: float = 0.30,
                   nsamp: int = 20000,
                   seed: int = 42) -> Tuple[pd.DataFrame, Dict[str, MBHEstimate]]:
    """
    Estimate M_BH from sigma and/or cluster mass and propagate to v_ej distributions
    for a set of semi-major axes. Returns a tidy DataFrame with medians and 16–84% ranges.
    """
    estimates = {}
    samples = {}

    if sigma_kms is not None:
        est_sigma = estimate_mbh_from_sigma(sigma_kms, method=method_sigma)
        estimates["from_sigma"] = est_sigma
        samples["from_sigma"] = sample_mbh(est_sigma, nsamp=nsamp, seed=seed)

    if M_cluster_Msun is not None:
        est_mcl = estimate_mbh_from_mcluster(M_cluster_Msun)
        estimates["from_Mcluster"] = est_mcl
        samples["from_Mcluster"] = sample_mbh(est_mcl, nsamp=nsamp, seed=seed+1)

    if not samples:
        raise ValueError("Provide at least sigma_kms or M_cluster_Msun.")

    m_tot = m1 + m2
    rows = []
    for key, mbh_draws in samples.items():
        # Propagate MBH uncertainty + additional v_ej fractional scatter (Gaussian in linear space)
        rng = np.random.default_rng(seed+2)
        for a in a_list_AU:
            vej_med = v_ej_median(a, m_tot, np.median(mbh_draws))
            # Sample v_ej by combining MBH variation (through ^(1/6)) and v scatter
            # Compute baseline v_ej per draw
            v_from_m = 460.0 * (a / 0.1) ** (-0.5) * (m_tot / 2.0) ** (1.0/3.0) * (mbh_draws / 1.0e3) ** (1.0/6.0)
            # Add extra fractional scatter
            v = v_from_m * rng.normal(loc=1.0, scale=v_scatter_frac, size=v_from_m.size)
            stats = summarize(v, qs=(0.16, 0.50, 0.84))
            rows.append({
                "Estimate": key,
                "a_AU": a,
                "v_ej_q16_kms": round(stats["q16"], 1),
                "v_ej_med_kms": round(stats["q50"], 1),
                "v_ej_q84_kms": round(stats["q84"], 1),
            })

    df = pd.DataFrame(rows).sort_values(["Estimate", "a_AU"]).reset_index(drop=True)
    return df, estimates

# -----------------------------
# Demo with 47 Tuc-like values
# From Lützgendorf+2013 Table 1: sigma ≈ 9.8 km/s; log M_tot ≈ 6.04 => M ≈ 1.10e6 Msun.
sigma_47tuc = 9.8
Mcl_47tuc = 10 ** 6.04

demo_df, demo_est = combine_to_vej(sigma_kms=sigma_47tuc, M_cluster_Msun=Mcl_47tuc,
                                   method_sigma="L13",
                                   a_list_AU=(0.05, 0.10, 0.20, 0.50),
                                   m1=0.8, m2=0.8,
                                   v_scatter_frac=0.30, nsamp=30000, seed=7)

# Prepare a readable summary DataFrame with the corresponding MBH medians
summary_rows = []
for key, est in demo_est.items():
    # Median MBH under the lognormal assumption equals 10^(log10_MBH) * exp(-0.5 * (ln10*sigma_dex)^2)
    mu_ln = est.log10_MBH * np.log(10.0)
    sig_ln = est.scatter_dex * np.log(10.0)
    median_mbh = np.exp(mu_ln - 0.5 * sig_ln**2)
    q16, q84 = np.exp(mu_ln + np.array([-1, 1]) * sig_ln)
    summary_rows.append({
        "Estimate": key,
        "Relation": est.source,
        "log10(M_BH/Msun)": round(est.log10_MBH, 3),
        "Scatter_dex": est.scatter_dex,
        "M_BH_med [Msun]": round(median_mbh, 0),
        "M_BH_q16 [Msun]": round(q16, 0),
        "M_BH_q84 [Msun]": round(q84, 0),
    })
mbh_summary = pd.DataFrame(summary_rows)

demo_df


,Estimate,a_AU,v_ej_q16_kms,v_ej_med_kms,v_ej_q84_kms
0,from_Mcluster,0.05,508.6,758.6,1050.6
1,from_Mcluster,0.10,359.3,536.3,746.7
2,from_Mcluster,0.20,254.7,378.2,525.9
3,from_Mcluster,0.50,160.9,239.7,331.2
4,from_sigma,0.05,427.4,627.3,845.2
5,from_sigma,0.10,303.9,442.6,598.4
6,from_sigma,0.20,214.8,312.5,422.2
7,from_sigma,0.50,135.8,197.9,266.6


In [63]:
mbh_summary

,Estimate,Relation,log10(M_BH/Msun),Scatter_dex,M_BH_med [Msun],M_BH_q16 [Msun],M_BH_q84 [Msun]
0,from_sigma,Lützgendorf+2013 EM (GC IMBH),3.114,0.36,922.0,567.0,2977.0
1,from_Mcluster,Lützgendorf+2013 EM (GC IMBH),3.620,0.49,2208.0,1350.0,12894.0
